In [ ]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from scipy.io import loadmat
import h5py
import functools

# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data import read_tfrecord_dataset

TEST_NAME = 'p4581'
DATA_DIR = '/misc/projects/criticalstress-ldrd/cwj/public/Lab_AE/data_lab_p4581/'

In [ ]:
if not os.path.exists('./Data/'):
  os.makedirs('./Data/')

# AE data info
Syncfile = loadmat(DATA_DIR+'ACTimeVector_4581.mat')
acsettings = loadmat(DATA_DIR+'p4581_AE.mat')

acTime = Syncfile['acTime'][0,:]
WFlength = int(acsettings['Nsamples'][0])
acTime_dt = np.mean(acTime[1:]-acTime[:-1])/WFlength
fs = 1.0/acTime_dt

seg_split = [
    # [2025.0, 2275.0, '2mpa_stepup', 220, 410], 
    [2325.0, 2525.0, '3mpa_stepup', 445, 600], 
    # [2575.0, 2875.0, '4mpa_stepup', 630, 860], 
    # [2925.0, 3175.0, '5mpa_stepup', 895, 1085], 
    # [3225.0, 3475.0, '6mpa_stepup', 1120, 1310], 
    # [3525.0, 3775.0, '7mpa_stepup', 1345, 1535], 
    # [3825.0, 4075.0, '8mpa_stepup', 1565, 1760], 
    # [4125.0, 4275.0, '7mpa_stepdown', 1790, 1910], 
    # [4325.0, 4475.0, '6mpa_stepdown', 1940, 2060], 
    # [4525.0, 4675.0, '5mpa_stepdown', 2090, 2210], 
    # [4725.0, 4825.0, '4mpa_stepdown', 2240, 2320], 
]

In [ ]:
from scipy import signal

for i_seg_split in seg_split:
    start_time = i_seg_split[0]
    end_time = i_seg_split[1]
    seg_name = i_seg_split[2]
    print('\n\n\n', seg_name, start_time, end_time)

    # Mech data
    with h5py.File(DATA_DIR+TEST_NAME+'_mechdata_'+seg_name+'.hdf5', 'r') as f:
        time_mu = f.get('time')[:].astype(np.float64)
        mu = f.get('mu')[:].astype(np.float32)

    # parameters data
    params_data = {}
    params_data['sample_freq']        = 1e3
    params_data['ae_sample_freq']     = fs
    params_data['tfrecord_data_dir']  = DATA_DIR+TEST_NAME+'_data_tfrecord_'+seg_name+'/'
    params_data['numCHR']             = len(acsettings['channels2save'][0])
    
    # AE dataset
    wf_ae_dataset = read_tfrecord_dataset(params_data)
    
    time_ae = []
    wf_ae = []
    ii = 0
    for ds in wf_ae_dataset:
        if ii%100000==0: print(ii)
        
        wf_ae_i = ds['wf_ae'][0,:,:].numpy()
        wf_ae_i = np.mean(wf_ae_i, axis=-1, keepdims=True)
        wf_ae_i = signal.resample(wf_ae_i, num=4096, axis=0)
        wf_ae_i = signal.decimate(wf_ae_i, 4, axis=0)
        wf_ae_i = signal.decimate(wf_ae_i, 2, axis=0)
        wf_ae.append(np.expand_dims(wf_ae_i, axis=0))

        t_win_i = ds['t_win'][0,:].numpy()
        time_ae.append(t_win_i[0:1])

        ii += 1

    time_ae = np.concatenate(time_ae, axis=0).astype(np.float32)
    wf_ae = np.concatenate(wf_ae, axis=0).astype(np.float32)

    print(time_ae.shape, wf_ae.shape)
    print(time_mu.shape, mu.shape)
    print(np.min(wf_ae), np.max(wf_ae))

    with h5py.File('./Data/p4581_seismic_data_'+seg_name+'.hdf5', 'w') as f:
      f.create_dataset('time', data = time_ae)
      f.create_dataset('wf_ae', data = wf_ae)

    with h5py.File('./Data/p4581_shear_data_'+seg_name+'.hdf5', 'w') as f:
      f.create_dataset('time', data = time_mu)
      f.create_dataset('mu', data = mu)